# Classification Project

By Ednalyn C. De Dios & Michael P. Moran

## Project Planning

### Goals
1. Explain what is driving customers to churn

### Deliverables
1. [ ] A report (jupyter notebook) containing analysis of what is driving customer churn
1. [ ] A csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn)
1. [ ] A single google slide that illustrates how your model works, including the features being used
    - Audience: senior leadership team
    - How were the values derived?
    - How likely is the model
        - to give a high probability of churn when churn doesn't occur,
        - to give a low probability of churn when churn occurs, and
        - to accurately predict churn.
1. [ ] A python script that prepares data such that it can be fed into your model
1. [ ] A README.md file that contains a link to your google slides presentation, and instructions for how to use your python script(s)

**Why are our customers churning?**

- Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
- Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
- Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
- If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

***Deliverables***

1. I will also need a report (ipynb) answering the question, "Why are our customers churning?" I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that month-to-month customers churn more, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight into why they are motivated to do so. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.
1. I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single goolgle slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.
1. Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.



### Data Dictionary & Domain Knowledge

### Hypotheses

### Thoughts & Questions

#### Questions
1. What is SLT?
#### From the boss
1. Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))
1. Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
1. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
1. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

## Prepare Environment

In [1]:
from env import host, user, password

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

## Acquisition

### Grab Data

1. Use the mysql connector to query telco_churn.customers. Assign the output of that query to the dataframe df. You want to include all the fields.

In [2]:
def get_db_url(
    hostname: str, username: str, password: str, db_name: str
) -> str:
    """
    return url for accessing a mysql database
    """
    return f"mysql+pymysql://{username}:{password}@{hostname}/{db_name}"


def get_sql_conn(hostname: str, username: str, password: str, db_name: str):
    """
    return a mysql connection object
    """
    return create_engine(get_db_url(host, user, password, db_name))


def df_from_sql(query: str, url: str) -> pd.DataFrame:
    """
    return a Pandas DataFrame resulting from a sql query
    """
    return pd.read_sql(query, url)


def get_telco_data() -> pd.DataFrame:
    db = "telco_churn"
    query = ("SELECT * "
             f"FROM customers;")
    url = get_db_url(host, user, password, db)
    return df_from_sql(query, url)

In [3]:
df = get_telco_data()

In [4]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.9,267.4,Yes


### Summarize Data

2. Write a function, peekatdata(dataframe), that takes a dataframe as input and computes and returns the following:

    - creates dataframe object head_df (df of the first 5 rows) and prints contents to screen
    - creates dataframe object tail_df (df of the last 5 rows) and prints contents to screen
    - creates tuple object shape_tuple (tuple of (nrows, ncols)) and prints tuple to screen
    - creates dataframe object describe_df (summary statistics of all numeric variables) and prints contents to screen.
    - prints to screen the information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.

In [5]:
def peekatdata(dataframe):
    head_df = dataframe.head()
    print(f"HEAD\n{head_df}", end="\n\n")

    tail_df = dataframe.tail()
    print(f"TAIL\n{tail_df}", end="\n\n")

    shape_tuple = dataframe.shape
    print(f"SHAPE: {shape_tuple}", end="\n\n")

    describe_df = dataframe.describe()
    print(f"DESCRIPTION\n{describe_df}", end="\n\n")

    print(f"INFORMATION")
    dataframe.info()

In [6]:
peekatdata(df)

HEAD
  customer_id  gender  senior_citizen partner dependents  tenure  \
0  0002-ORFBO  Female               0     Yes        Yes       9   
1  0003-MKNFE    Male               0      No         No       9   
2  0004-TLHLJ    Male               0      No         No       4   
3  0011-IGKFF    Male               1     Yes         No      13   
4  0013-EXCHZ  Female               1     Yes         No       3   

  phone_service multiple_lines  internet_service_type_id online_security  \
0           Yes             No                         1              No   
1           Yes            Yes                         1              No   
2           Yes             No                         2              No   
3           Yes             No                         2              No   
4           Yes             No                         2              No   

   ...  device_protection tech_support streaming_tv streaming_movies  \
0  ...                 No          Yes          Yes      

## Data Prep

### TODO
- [ ] multiple_lines has "No phone service" for the last row, which is not the same as yes/no that other columns have.
- [ ] convert total_charges column to float. it is an object after reading from sql database

1. Write a function, df_value_counts(dataframe), that takes a dataframe as input and computes and returns the values by frequency for each variable. Use the rule of thumb for your logic on whether or not to use the bins argument. The function will use a for loop and an in statement.

In [7]:
def df_value_counts(dataframe):
    for col in df.columns:
        n = df[col].unique().shape[0]
        col_bins = min(n, 10)
        print(f"{col}:")
        if df[col].dtype in ['int64', 'float64'] and n > 10:
            print(df[col].value_counts(bins=col_bins, sort=False))
        else:
            print(df[col].value_counts())
        print("\n")

In [8]:
df_value_counts(df)

customer_id:
3071-MVJCD    1
2002-MZHWP    1
8707-HOEDG    1
4821-WQOYN    1
2265-CYWIV    1
2890-WFBHU    1
1550-LOAHA    1
8495-LJDFO    1
4998-IKFSE    1
6769-DCQLI    1
6128-AQBMT    1
2654-VBVPB    1
4855-SNKMY    1
4238-HFHSN    1
1163-ONYEY    1
6549-YMFAW    1
0531-XBKMM    1
5859-HZYLF    1
3550-SAHFP    1
1689-YQBYY    1
2348-KCJLT    1
6941-PMGEP    1
2869-ADAWR    1
3750-CKVKH    1
1247-QBVSH    1
5835-BEQEU    1
5909-ECHUI    1
7765-LWVVH    1
1257-SXUXQ    1
2294-DMMUS    1
             ..
2599-CIPQE    1
2580-ASVVY    1
6959-UWKHF    1
2504-DSHIH    1
6374-AFWOX    1
9030-QGZNL    1
0675-NCDYU    1
6701-YVNQG    1
2968-SSGAA    1
3410-MHHUM    1
2207-QPJED    1
3078-ZKNTS    1
2324-AALNO    1
0927-CNGRH    1
4859-ZSRDZ    1
9150-KPBJQ    1
2030-BTZRO    1
4660-IRIBM    1
3934-HXCFZ    1
1950-KSVVJ    1
4808-YNLEU    1
1090-PYKCI    1
1092-WPIVQ    1
6608-QQLVK    1
7945-HLKEA    1
8559-WNQZS    1
2277-VWCNI    1
4029-HPFVY    1
8095-WANWK    1
1767-CJKBA    1
Name: custo

- customer_id has no duplicates
- gender is about even
- customers are mostly not seniors
- about equally split along single/partner
- most customers do not have dependents
- there are many new and many old customers
- overwhelming majority have phone service
- closely split along multiple_lines
- overwhelming majority have internet service
    - more have fiber than DSL
    - most do not have online_security
    - most do not have online_backup
    - most do not have device_protection
    - most do not have tech_support
- billing
    - about evenly split along streaming_tv
    - about evenly splot along streaming_movies
    - most customers are month-to-month
    - most customers are paperless billing
    - most customers pay by some form of check
    - many customers pay less than $30
    - most are pay between 45 and 110 per month
- most have not churned
    - about 1900 have
    

### Handle Missing Values

2. Missing Values:

    - Write a function, that returns a dataframe of the column name and the number of missing values and the percentage of missing values (missing records/total records) for each of the columns that have > 0 missing values.

   - Document your takeaways. For each variable:

        - should you remove the observations with a missing value for that variable?
        - should you remove the variable altogether?
        - is missing equivalent to 0 (or some other constant value) in the specific case of this variable?
        - should you replace the missing values with a value it is most likely to represent (e.g. Are the missing values a result of data integrity issues and should be replaced by the most likely value?)
        - Handle the missing values in the way you recommended above.

In [9]:
def df_missing_vals(dataframe):
    null_count = dataframe.isnull().sum()
    null_percentage = (null_count / dataframe.shape[0]) * 100
    empty_count = pd.Series(((dataframe == " ") | (dataframe == "")).sum())
    return pd.DataFrame({"nmissing": null_count, "percentage": null_percentage, "nempty": empty_count})

# test 
# print(df_missing_vals(pd.DataFrame({"col1": [np.nan, 1, "", np.nan, np.nan], "col2": [2, "", 4, np.nan, 4]})))

In [10]:
print(df_missing_vals(df))

                          nmissing  percentage  nempty
customer_id                      0         0.0       0
gender                           0         0.0       0
senior_citizen                   0         0.0       0
partner                          0         0.0       0
dependents                       0         0.0       0
tenure                           0         0.0       0
phone_service                    0         0.0       0
multiple_lines                   0         0.0       0
internet_service_type_id         0         0.0       0
online_security                  0         0.0       0
online_backup                    0         0.0       0
device_protection                0         0.0       0
tech_support                     0         0.0       0
streaming_tv                     0         0.0       0
streaming_movies                 0         0.0       0
contract_type_id                 0         0.0       0
paperless_billing                0         0.0       0
payment_ty

- Document your takeaways. For each variable:
    - Me: No columns have NaNs
    - Me: total_charges has 11 observations with a " " or "" value
        - let's replace the 11 with their monthly_charges * tenure
    - should you remove the observations with a missing value for that variable?
    - should you remove the variable altogether?
    - is missing equivalent to 0 (or some other constant value) in the specific case of this variable?
    - should you replace the missing values with a value it is most likely to represent (e.g. Are the missing values a result of data integrity issues and should be replaced by the most likely value?)

Handle the missing values in the way you recommended above.

In [11]:
df['total_charges'] = np.where(df['total_charges'] == ' ', (df.monthly_charges * df.tenure), df['total_charges'])
print(df_missing_vals(df))
df.head(10)

                          nmissing  percentage  nempty
customer_id                      0         0.0       0
gender                           0         0.0       0
senior_citizen                   0         0.0       0
partner                          0         0.0       0
dependents                       0         0.0       0
tenure                           0         0.0       0
phone_service                    0         0.0       0
multiple_lines                   0         0.0       0
internet_service_type_id         0         0.0       0
online_security                  0         0.0       0
online_backup                    0         0.0       0
device_protection                0         0.0       0
tech_support                     0         0.0       0
streaming_tv                     0         0.0       0
streaming_movies                 0         0.0       0
contract_type_id                 0         0.0       0
paperless_billing                0         0.0       0
payment_ty

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.60,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.90,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.90,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.00,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.90,267.4,Yes
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,No,Yes,Yes,Yes,1,Yes,4,69.40,571.45,No
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,No
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,No,Yes,No,No,3,Yes,4,84.65,5377.8,No
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,No,1,Yes,1,48.20,340.35,No
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,No


3. Transform churn such that "yes" = 1 and "no" = 0

In [12]:
df['churn'] = df['churn'].map( {'No': 0, 'Yes': 1} ).astype(int)
df.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.60,593.3,0
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.90,542.4,0
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.90,280.85,1
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,Yes,No,Yes,Yes,1,Yes,1,98.00,1237.85,1
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,No,Yes,Yes,No,1,Yes,2,83.90,267.4,1
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,No,Yes,Yes,Yes,1,Yes,4,69.40,571.45,0
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,0
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,No,Yes,No,No,3,Yes,4,84.65,5377.8,0
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,No,1,Yes,1,48.20,340.35,0
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,0


4. Compute a new feature, tenure_year, that is a result of translating tenure from months to years.

In [13]:
df['tenure_year'] = df.tenure / 12
df.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,Yes,Yes,No,2,Yes,2,65.60,593.3,0,0.750000
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,Yes,1,No,2,59.90,542.4,0,0.750000
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,No,No,No,1,Yes,1,73.90,280.85,1,0.333333
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,No,Yes,Yes,1,Yes,1,98.00,1237.85,1,1.083333
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,Yes,Yes,No,1,Yes,2,83.90,267.4,1,0.250000
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,Yes,Yes,Yes,1,Yes,4,69.40,571.45,0,0.750000
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,Yes,Yes,Yes,3,Yes,3,109.70,7904.25,0,5.916667
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,Yes,No,No,3,Yes,4,84.65,5377.8,0,5.250000
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,No,1,Yes,1,48.20,340.35,0,0.583333
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,Yes,Yes,3,Yes,2,90.45,5957.9,0,5.416667


5. Figure out a way to capture the information contained in phone_service and multiple_lines into a single variable of dtype int. Write a function that will transform the data and place in a new column phone_id in df_sql. Be sure you have documented your function and logic well.

In [14]:
def add_phone_id(df):
    df_temp = df
    df_temp['phone_id'] = np.where(df['phone_service'] == 'No', 0, 9)
    df_temp['phone_id'] = np.where((df['phone_service'] == 'Yes') & (df['multiple_lines'] == 'No'), 1, df_temp['phone_id'])
    df_temp['phone_id'] = np.where((df['phone_service'] == 'Yes') & (df['multiple_lines'] == 'Yes'), 2, df_temp['phone_id'])
    return df_temp

df_sql = add_phone_id(df)
print(df_sql['phone_id'].dtype)
df.tail(10)

int64


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year,phone_id
7033,9975-SKRNR,Male,0,No,No,1,Yes,No,3,No internet service,...,No internet service,No internet service,1,No,2,18.90,18.9,0,0.083333,1
7034,9978-HYCIN,Male,1,Yes,Yes,47,Yes,No,2,No,...,Yes,No,2,Yes,3,84.95,4018.05,0,3.916667,1
7035,9979-RGMZT,Female,0,No,No,7,Yes,No,2,No,...,Yes,Yes,2,Yes,2,94.05,633.45,0,0.583333,1
7036,9985-MWVIX,Female,0,No,No,1,Yes,No,2,No,...,No,No,1,Yes,2,70.15,70.15,1,0.083333,1
7037,9986-BONCE,Female,0,No,No,4,Yes,No,3,No internet service,...,No internet service,No internet service,1,No,3,20.95,85.5,1,0.333333,1
7038,9987-LUTYD,Female,0,No,No,13,Yes,No,1,Yes,...,No,No,2,No,2,55.15,742.9,0,1.083333,1
7039,9992-RRAMN,Male,0,Yes,No,22,Yes,Yes,2,No,...,No,Yes,1,Yes,1,85.10,1873.7,1,1.833333,2
7040,9992-UJOEL,Male,0,No,No,2,Yes,No,1,No,...,No,No,1,Yes,2,50.30,92.75,0,0.166667,1
7041,9993-LHIEB,Male,0,Yes,Yes,67,Yes,No,1,Yes,...,No,Yes,3,No,2,67.85,4627.65,0,5.583333,1
7042,9995-HOTOH,Male,0,Yes,Yes,63,No,No phone service,1,Yes,...,Yes,Yes,3,No,1,59.00,3707.6,0,5.250000,0


6. Figure out a way to capture the information contained in dependents and partner into a single variable of dtype int. Transform the data and place in a new column household_type_id in df_sql. Be sure you have documented your function and logic well.

In [15]:
# 0 = no partner, no dependents
# 1 = has partner, has dependents
# 2 = has partner, no dependents
# 3 = no partner, has dependents
def add_household_type(df):
    df_temp = df
    df_temp['household_type'] = np.where((df['partner'] == 'No') & (df['dependents'] == 'No'), 0, 9)
    df_temp['household_type'] = np.where((df['partner'] == 'Yes') & (df['dependents'] == 'Yes'), 1, df_temp['household_type'])
    df_temp['household_type'] = np.where((df['partner'] == 'Yes') & (df['dependents'] == 'No'), 2, df_temp['household_type'])
    df_temp['household_type'] = np.where((df['partner'] == 'No') & (df['dependents'] == 'Yes'), 3, df_temp['household_type'])
    return df_temp

df_sql = add_household_type(df)
print(df_sql['household_type'].dtype)
df.tail(100)

int64


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year,phone_id,household_type
6943,9846-GKXAS,Female,0,No,No,9,Yes,No,2,No,...,Yes,1,Yes,3,90.80,809.75,1,0.750000,1,0
6944,9847-HNVGP,Male,0,No,No,17,Yes,No,3,No internet service,...,No internet service,2,No,2,19.95,310.6,0,1.416667,1,0
6945,9848-JQJTX,Male,0,No,No,72,Yes,Yes,2,No,...,Yes,3,Yes,3,100.90,7459.05,0,6.000000,2,0
6946,9850-OWRHQ,Female,0,Yes,No,3,Yes,Yes,2,No,...,No,1,Yes,4,74.10,228,1,0.250000,2,2
6947,9851-KIELU,Male,0,No,No,10,Yes,No,2,Yes,...,Yes,1,Yes,1,110.10,1043.3,1,0.833333,1,0
6948,9851-QXEEQ,Male,0,No,No,41,Yes,Yes,2,No,...,Yes,1,Yes,1,104.70,4346.4,1,3.416667,2,0
6949,9853-JFZDU,Female,0,Yes,No,35,Yes,No,1,No,...,No,1,No,1,56.85,1861.1,0,2.916667,1,2
6950,9860-LISIZ,Female,0,No,No,34,Yes,No,3,No internet service,...,No internet service,1,No,2,20.35,673.2,0,2.833333,1,0
6951,9861-PDSZP,Female,0,No,No,72,Yes,Yes,2,No,...,Yes,3,Yes,4,108.05,7806.6,0,6.000000,2,0
6952,9862-KJTYK,Male,0,No,Yes,19,No,No phone service,1,No,...,No,1,Yes,4,25.35,566.1,0,1.583333,0,3


7. Figure out a way to capture the information contained in streaming_tv and streaming_movies into a single variable of dtype int. Transform the data and place in a new column streaming_services in df_sql. Be sure you have documented your function and logic well.

In [16]:
# 0 = no streaming_tv, no streaming_movies
# 1 = has streaming_tv, has streaming_movies
# 2 = has streaming_tv, no streaming_movies
# 3 = no streaming_tv, has streaming_movies
# 4 = no internet service
def add_streaming_services(df):
    df_temp = df
    df_temp['streaming_services'] = np.where((df['streaming_tv'] == 'No') & (df['streaming_movies'] == 'No'), 0, 9)
    df_temp['streaming_services'] = np.where((df['streaming_tv'] == 'Yes') & (df['streaming_movies'] == 'Yes'), 1, df_temp['streaming_services'])
    df_temp['streaming_services'] = np.where((df['streaming_tv'] == 'Yes') & (df['streaming_movies'] == 'No'), 2, df_temp['streaming_services'])
    df_temp['streaming_services'] = np.where((df['streaming_tv'] == 'No') & (df['streaming_movies'] == 'Yes'), 3, df_temp['streaming_services'])
    df_temp['streaming_services'] = np.where((df['streaming_tv'] == 'No internet service') & (df['streaming_movies'] == 'No internet service'), 4, df_temp['streaming_services'])
    return df_temp

df_sql = add_streaming_services(df)
print(df_sql['streaming_services'].dtype)
df.tail(10)

int64


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year,phone_id,household_type,streaming_services
7033,9975-SKRNR,Male,0,No,No,1,Yes,No,3,No internet service,...,1,No,2,18.90,18.9,0,0.083333,1,0,4
7034,9978-HYCIN,Male,1,Yes,Yes,47,Yes,No,2,No,...,2,Yes,3,84.95,4018.05,0,3.916667,1,1,2
7035,9979-RGMZT,Female,0,No,No,7,Yes,No,2,No,...,2,Yes,2,94.05,633.45,0,0.583333,1,0,1
7036,9985-MWVIX,Female,0,No,No,1,Yes,No,2,No,...,1,Yes,2,70.15,70.15,1,0.083333,1,0,0
7037,9986-BONCE,Female,0,No,No,4,Yes,No,3,No internet service,...,1,No,3,20.95,85.5,1,0.333333,1,0,4
7038,9987-LUTYD,Female,0,No,No,13,Yes,No,1,Yes,...,2,No,2,55.15,742.9,0,1.083333,1,0,0
7039,9992-RRAMN,Male,0,Yes,No,22,Yes,Yes,2,No,...,1,Yes,1,85.10,1873.7,1,1.833333,2,2,3
7040,9992-UJOEL,Male,0,No,No,2,Yes,No,1,No,...,1,Yes,2,50.30,92.75,0,0.166667,1,0,0
7041,9993-LHIEB,Male,0,Yes,Yes,67,Yes,No,1,Yes,...,3,No,2,67.85,4627.65,0,5.583333,1,1,3
7042,9995-HOTOH,Male,0,Yes,Yes,63,No,No phone service,1,Yes,...,3,No,1,59.00,3707.6,0,5.250000,0,1,1


8. Figure out a way to capture the information contained in online_security and online_backup into a single variable of dtype int. Transform the data and place in a new column online_security_backup in df_sql. Be sure you have documented your function and logic well.

In [17]:
# 0 = no online_security, no online_backup
# 1 = has online_security, has online_backup
# 2 = has online_security, no online_backup
# 3 = no online_security, has online_backup
# 4 = no internet service
def add_online_security_backup(df):
    df_temp = df
    df_temp['online_security_backup'] = np.where((df['online_security'] == 'No') & (df['online_backup'] == 'No'), 0, 9)
    df_temp['online_security_backup'] = np.where((df['online_security'] == 'Yes') & (df['online_backup'] == 'Yes'), 1, df_temp['online_security_backup'])
    df_temp['online_security_backup'] = np.where((df['online_security'] == 'Yes') & (df['online_backup'] == 'No'), 2, df_temp['online_security_backup'])
    df_temp['online_security_backup'] = np.where((df['online_security'] == 'No') & (df['online_backup'] == 'Yes'), 3, df_temp['online_security_backup'])
    df_temp['online_security_backup'] = np.where((df['online_security'] == 'No internet service') & (df['online_backup'] == 'No internet service'), 4, df_temp['online_security_backup'])
    return df_temp

df_sql = add_online_security_backup(df)
print(df_sql['online_security_backup'].dtype)
df.tail(10)

int64


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,tenure_year,phone_id,household_type,streaming_services,online_security_backup
7033,9975-SKRNR,Male,0,No,No,1,Yes,No,3,No internet service,...,No,2,18.90,18.9,0,0.083333,1,0,4,4
7034,9978-HYCIN,Male,1,Yes,Yes,47,Yes,No,2,No,...,Yes,3,84.95,4018.05,0,3.916667,1,1,2,3
7035,9979-RGMZT,Female,0,No,No,7,Yes,No,2,No,...,Yes,2,94.05,633.45,0,0.583333,1,0,1,3
7036,9985-MWVIX,Female,0,No,No,1,Yes,No,2,No,...,Yes,2,70.15,70.15,1,0.083333,1,0,0,0
7037,9986-BONCE,Female,0,No,No,4,Yes,No,3,No internet service,...,No,3,20.95,85.5,1,0.333333,1,0,4,4
7038,9987-LUTYD,Female,0,No,No,13,Yes,No,1,Yes,...,No,2,55.15,742.9,0,1.083333,1,0,0,2
7039,9992-RRAMN,Male,0,Yes,No,22,Yes,Yes,2,No,...,Yes,1,85.10,1873.7,1,1.833333,2,2,3,0
7040,9992-UJOEL,Male,0,No,No,2,Yes,No,1,No,...,Yes,2,50.30,92.75,0,0.166667,1,0,0,3
7041,9993-LHIEB,Male,0,Yes,Yes,67,Yes,No,1,Yes,...,No,2,67.85,4627.65,0,5.583333,1,1,3,2
7042,9995-HOTOH,Male,0,Yes,Yes,63,No,No phone service,1,Yes,...,No,1,59.00,3707.6,0,5.250000,0,1,1,1


9. Data Split

    - Split data into train (70%) & test (30%) samples. You should end with 2 data frames: train_df and test_df

In [18]:
print(df_missing_vals(df))
df_value_counts(df)

                          nmissing  percentage  nempty
customer_id                      0         0.0       0
gender                           0         0.0       0
senior_citizen                   0         0.0       0
partner                          0         0.0       0
dependents                       0         0.0       0
tenure                           0         0.0       0
phone_service                    0         0.0       0
multiple_lines                   0         0.0       0
internet_service_type_id         0         0.0       0
online_security                  0         0.0       0
online_backup                    0         0.0       0
device_protection                0         0.0       0
tech_support                     0         0.0       0
streaming_tv                     0         0.0       0
streaming_movies                 0         0.0       0
contract_type_id                 0         0.0       0
paperless_billing                0         0.0       0
payment_ty

In [19]:
X = df_sql[['gender', 'senior_citizen', 'internet_service_type_id', 'device_protection', 'tech_support', 'contract_type_id', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges', 'tenure_year', 'phone_id', 'household_type', 'streaming_services', 'online_security_backup']]
y = df_sql[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

X_train.head()

,gender,senior_citizen,internet_service_type_id,device_protection,tech_support,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,tenure_year,phone_id,household_type,streaming_services,online_security_backup
1479,Male,0,2,No,No,1,Yes,3,79.95,1043.4,1.000000,1,0,3,0
2377,Female,1,1,No,Yes,1,Yes,2,63.55,1381.8,1.833333,2,2,0,1
6613,Female,0,3,No internet service,No internet service,3,No,4,24.60,1678.05,5.750000,2,0,4,4
6468,Female,0,3,No internet service,No internet service,3,No,3,25.00,1753,5.916667,2,2,4,4
2668,Female,1,2,No,No,1,Yes,1,95.05,1290,1.083333,2,0,1,0


10. Variable Encoding

    - Write an encoder (fit and transform on train_df) for each non-numeric variable. Use that encoder object to transform on test_df

In [20]:
# encoder = LabelEncoder()

# print(df_sql.gender.value_counts(dropna=False))
# print(df_sql.device_protection.value_counts(dropna=False))
# print(df_sql.tech_support.value_counts(dropna=False))
# print(df_sql.paperless_billing.value_counts(dropna=False))

# encoder.fit(df_sql.gender)
# df_sql.gender = encoder.transform(df_sql.gender)
# print(df_sql.gender.head(10))

# encoder.fit(df_sql.device_protection)
# df_sql.device_protection = encoder.transform(df_sql.device_protection)
# print(df_sql.device_protection.head(10))

# encoder.fit(df_sql.tech_support)
# df_sql.tech_support = encoder.transform(df_sql.tech_support)
# print(df_sql.tech_support.head(10))

# encoder.fit(df_sql.paperless_billing)
# df_sql.paperless_billing = encoder.transform(df_sql.paperless_billing)
# print(df_sql.paperless_billing.head(10))

11. Numeric Scaling

    - Fit a min_max_scaler to train_df. Transform monthly_charges and total_charges variables in train_df using the scaler. Then use the scaler object to transform test_df.

## Data Exploration

### Deliverable

*I will also need a report (ipynb) answering the question, "Why are our customers churning?" I want to see the analysis you did to answer my questions and lead to your findings. Please clearly call out the questions and answers you are analyzing. E.g. If you find that month-to-month customers churn more, I won't be surprised, but I am not getting rid of that plan. The fact that they churn is not because they can, it's because they can and they are motivated to do so. I want some insight into why they are motivated to do so. I realize you will not be able to do a full causal experiment, but I hope to see some solid evidence of your conclusions.*

1. Could the month in which they signed up influence churn? i.e. if a cohort is identified by tenure, is there a cohort or cohorts who have a higher rate of churn than other cohorts? (Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers))

2. Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?

3. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?

4. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

5. Controlling for services (phone_id, internet_service_type_id, online_security_backup, device_protection, tech_support, and contract_type_id), is the mean monthly_charges of those who have churned significantly different from that of those who have not churned?

6. How much of monthly_charges can be explained by internet_service_type? (hint: correlation test). State your hypotheses and your conclusion clearly.

7. How much of monthly_charges can be explained by internet_service_type + phone service type (0, 1, or multiple lines). State your hypotheses and your conclusion clearly.

8. Create visualizations exploring the interactions of variables (independent with independent and independent with dependent). The goal is to identify features that are related to churn, identify any data integrity issues, understand 'how the data works', e.g. we may find that all who have online services also have device protection. In that case, we don't need both of those. (The visualizations done in your analysis for questions 1-5 count towards the requirements below)

    - Each independent variable (except for customer_id) must be visualized in at least two plots, and at least 1 of those compares the independent variable with the dependent variable.

    - For each plot where x and y are independent variables, add a third dimension (where possible), of churn represented by color.

    - Use subplots when plotting the same type of chart but with different variables.

    - Adjust the axes as necessary to extract information from the visualizations (adjusting the x & y limits, setting the scale where needed, etc.)

    - Add annotations to at least 5 plots with a key takeaway from that plot.

    - Use plots from matplotlib, pandas and seaborn.

    - Use each of the following:

        - sns.heatmap
        - pd.crosstab (with color)
        - pd.scatter_matrix
        - sns.barplot
        - sns.swarmplot
        - sns.pairplot
        - sns.jointplot
        - sns.relplot or plt.scatter
        - sns.distplot or plt.hist
        - sns.boxplot
        - plt.plot
        
    - Use at least one more type of plot that is not included in the list above.
    

9. What can you say about each variable's relationship to churn, based on your initial exploration? If there appears to be some sort of interaction or correlation, assume there is no causal relationship and brainstorm (and document) ideas on reasons there could be correlation.

    - phone_id
    - internet_service_type_id
    - online_security_backup
    - device_protection
    - tech_support
    - contract_type_id
    - senior_citizen
    - tenure
    - tenure_year
    - monthly_charges
    - total_charges
    - payment_type_id
    - paperless_billing
    - contract_type_id
    - gender
    
   
10. Summarize your conclusions, provide clear answers to the specific questions, and summarize any takeaways/action plan from the work above.

## Modeling

1. Feature Selection: Are there any variables that seem to provide limited to no additional information? If so, remove those and assign the new limited dataframe to train_reduced

2. Train (fit, transform, evaluate) a logistic regression model varying your meta-parameters.

3. Compare evaluation metrics across all the models, and select the best performing model.

4. Test the final model (transform, evaluate) on your out-of-sample data (test_df). Summarize the performance. Interpret your results.

# Delivery

1. I will need you to deliver to me a csv with the customer_id, probability of churn, and the prediction of churn (1=churn, 0=not_churn). I would also like a single google slide that illustrates how your model works, including the features being used, so that I can deliver this to the SLT when they come with questions about how these values were derived. Please make sure you include how likely your model is to give a high probability of churn when churn doesn't occur, to give a low probability of churn when churn occurs, and to accurately predict churn.

1. Finally, our development team will need a .py file that will take in a new dataset, (in the exact same form of the one you acquired from telco_churn.customers) and perform all the transformations necessary to run the model you have developed on this new dataset to provide probabilities and predictions.